In [1]:
import tensorflow as tf
import tensorlayer as tl

/home/jing/anaconda2/lib/python2.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [2]:
graph = tf.get_default_graph()
config = tf.ConfigProto()
session = tf.Session(graph=graph, config=config)

In [3]:
obs_shape = [3,512,480, 640,15] #should be NHWC
batch_size = 1
image = tf.placeholder(tf.float32, [batch_size,obs_shape[2],obs_shape[3],obs_shape[4]], 'image_ph')

#forward layers
imagenet = tl.layers.InputLayer(image, name='image_input')
imagenet = tl.layers.Conv2dLayer(imagenet, act=tf.nn.relu, shape=(7, 7, obs_shape[4], 64), strides=(1, 2, 2, 1),use_cudnn_on_gpu=True, name='cnn1')
imagenet = tl.layers.Conv2dLayer(imagenet, act=tf.nn.relu, shape=(5, 5, 64, 128), strides=(1, 2, 2, 1),use_cudnn_on_gpu=True, name='cnn2')
imagenetout2 = imagenet.outputs
print(imagenetout2.shape)

imagenet = tl.layers.Conv2dLayer(imagenet, act=tf.nn.relu, shape=(5, 5, 128, 256), strides=(1, 2, 2, 1),use_cudnn_on_gpu=True, name='cnn3')
imagenet = tl.layers.Conv2dLayer(imagenet, act=tf.nn.relu, shape=(3, 3, 256, 256), strides=(1, 1, 1, 1),use_cudnn_on_gpu=True, name='cnn31')
imagenetout3 = imagenet.outputs
print(imagenetout3.shape)

imagenet = tl.layers.Conv2dLayer(imagenet, act=tf.nn.relu, shape=(3, 3, 256, 512), strides=(1, 2, 2, 1),use_cudnn_on_gpu=True, name='cnn4')
imagenet = tl.layers.Conv2dLayer(imagenet, act=tf.nn.relu, shape=(3, 3, 512, 512), strides=(1, 1, 1, 1),use_cudnn_on_gpu=True, name='cnn41')
imagenetout4 = imagenet.outputs
print(imagenetout4.shape)

imagenet = tl.layers.Conv2dLayer(imagenet, act=tf.nn.relu, shape=(3, 3, 512, 512), strides=(1, 2, 2, 1),use_cudnn_on_gpu=True, name='cnn5')
imagenet = tl.layers.Conv2dLayer(imagenet, act=tf.nn.relu, shape=(3, 3, 512, 512), strides=(1, 1, 1, 1),use_cudnn_on_gpu=True, name='cnn51')
imagenetout5 = imagenet.outputs
print(imagenetout5.shape)

imagenet = tl.layers.Conv2dLayer(imagenet, act=tf.nn.relu, shape=(3, 3, 512, 1024), strides=(1, 2, 2, 1),use_cudnn_on_gpu=True, name='cnn6')
imagenet = tl.layers.Conv2dLayer(imagenet, act=tf.nn.relu, shape=(3, 3, 1024, 1024), strides=(1, 1, 1, 1),use_cudnn_on_gpu=True, name='cnn61')
imagenetout6 = imagenet.outputs  #used in training
print(imagenetout6.shape)


#refine layers
imagenet = tl.layers.DeConv2dLayer(imagenet, act=tf.nn.relu,shape=(1, 1, 512, 1024),output_shape=(batch_size, 20, 15, 512), strides=(1, 2, 2, 1), name='decnn5')
imagenetoutu5 = imagenet.outputs
print(imagenetoutu5.shape)

imagenet2 = tl.layers.InputLayer(tf.concat([imagenetoutu5,imagenetout5], axis=3), name='Irefinein1')
imagenet3 = tl.layers.Conv2dLayer(imagenet2, act=tf.nn.relu, shape=(5, 5, 1024, 3), strides=(1, 1, 1, 1), use_cudnn_on_gpu=True, name='Iflow5')
imagenet3 = tl.layers.UpSampling2dLayer(imagenet3,size=(2,2), method=0, name='Iunsample4')
imagenetoutf5u = imagenet3.outputs
imagenet4 = tl.layers.DeConv2dLayer(imagenet2, act=tf.nn.relu,shape=(1, 1, 256, 1024),output_shape=(batch_size, 40, 30, 256), strides=(1, 2, 2, 1), name='decnn4')
imagenetoutu4 = imagenet4.outputs

imagenet5 = tl.layers.InputLayer(tf.concat([imagenetoutu4,imagenetout4,imagenetoutf5u], axis=3), name='Irefinein2')
imagenet6 = tl.layers.Conv2dLayer(imagenet5, act=tf.nn.relu, shape=(5, 5, 771, 3), strides=(1, 1, 1, 1), use_cudnn_on_gpu=True, name='Iflow4')
imagenet6 = tl.layers.UpSampling2dLayer(imagenet6,size=(2,2), method=0, name='Iunsample3')
imagenetoutf4u = imagenet6.outputs
imagenet7 = tl.layers.DeConv2dLayer(imagenet5, act=tf.nn.relu,shape=(1, 1, 128, 771),output_shape=(batch_size, 80, 60, 128), strides=(1, 2, 2, 1), name='decnn3')
imagenetoutu3 = imagenet7.outputs

imagenet8 = tl.layers.InputLayer(tf.concat([imagenetoutu3,imagenetout3,imagenetoutf4u], axis=3), name='Irefinein3')
imagenet9 = tl.layers.Conv2dLayer(imagenet8, act=tf.nn.relu, shape=(5, 5, 387, 3), strides=(1, 1, 1, 1), use_cudnn_on_gpu=True, name='Iflow3')
imagenet9 = tl.layers.UpSampling2dLayer(imagenet9,size=(2,2), method=0, name='Iunsample2')
imagenetoutf3u = imagenet9.outputs
imagenet10 = tl.layers.DeConv2dLayer(imagenet8, act=tf.nn.relu,shape=(1, 1, 64, 387),output_shape=(batch_size, 160, 120, 64), strides=(1, 2, 2, 1), name='decnn2')
imagenetoutu2 = imagenet10.outputs

imagenet11 = tl.layers.InputLayer(tf.concat([imagenetoutu2,imagenetout2,imagenetoutf3u], axis=3), name='Irefinein4')
imagenet12 = tl.layers.Conv2dLayer(imagenet11, act=tf.nn.relu, shape=(5, 5, 195, 3), strides=(1, 1, 1, 1), use_cudnn_on_gpu=True, name='Iflow2')
imagenetoutf2u = imagenet12.outputs


[TL] InputLayer  image_input: (1, 640, 480, 15)
[TL] Conv2dLayer cnn1: shape: (7, 7, 15, 64) strides: (1, 2, 2, 1) pad: SAME act: relu
Instructions for updating:
Colocations handled automatically by placer.
[TL] Conv2dLayer cnn2: shape: (5, 5, 64, 128) strides: (1, 2, 2, 1) pad: SAME act: relu
(1, 160, 120, 128)
[TL] Conv2dLayer cnn3: shape: (5, 5, 128, 256) strides: (1, 2, 2, 1) pad: SAME act: relu
[TL] Conv2dLayer cnn31: shape: (3, 3, 256, 256) strides: (1, 1, 1, 1) pad: SAME act: relu
(1, 80, 60, 256)
[TL] Conv2dLayer cnn4: shape: (3, 3, 256, 512) strides: (1, 2, 2, 1) pad: SAME act: relu
[TL] Conv2dLayer cnn41: shape: (3, 3, 512, 512) strides: (1, 1, 1, 1) pad: SAME act: relu
(1, 40, 30, 512)
[TL] Conv2dLayer cnn5: shape: (3, 3, 512, 512) strides: (1, 2, 2, 1) pad: SAME act: relu
[TL] Conv2dLayer cnn51: shape: (3, 3, 512, 512) strides: (1, 1, 1, 1) pad: SAME act: relu
(1, 20, 15, 512)
[TL] Conv2dLayer cnn6: shape: (3, 3, 512, 1024) strides: (1, 2, 2, 1) pad: SAME act: relu
[TL] Con

In [3]:

obs_shape = [3,512,640, 480,3,5] #should be NHWC
batch_size = 1
imagetype = 'image'

image    = tf.placeholder(tf.float32, [None,obs_shape[5],obs_shape[2],obs_shape[3],obs_shape[4]], 'image_ph')

def keras_block(imagenetx):
    imagenet = tf.keras.layers.ConvLSTM2D(filters=16,kernel_size=[7,7],strides=[2,2],padding='same',activation=tf.nn.relu,return_sequences=True)(imagenetx)
    imagenet = tf.keras.layers.BatchNormalization()(imagenet)
    imagenet = tf.keras.layers.ConvLSTM2D(filters=32,kernel_size=[5,5],strides=[2,2],padding='same',activation=tf.nn.relu,return_sequences=True)(imagenet)
    imagenet = tf.keras.layers.BatchNormalization()(imagenet)
    imagenet = tf.keras.layers.ConvLSTM2D(filters=64,kernel_size=[3,3],strides=[2,2],padding='same',activation=tf.nn.relu,return_sequences=True)(imagenet)
    imagenet = tf.keras.layers.BatchNormalization()(imagenet)
    imagenet = tf.keras.layers.ConvLSTM2D(filters=64,kernel_size=[3,3],strides=[1,1],padding='same',activation=tf.nn.relu,return_sequences=True)(imagenet)
    imagenet = tf.keras.layers.BatchNormalization()(imagenet)
    imagenet = tf.keras.layers.Conv3D(filters=1, kernel_size=(3, 3, 3),activation=tf.nn.relu,padding='same')(imagenet)
    return imagenet

imagenet = tl.layers.InputLayer(image, name='image_input')
imagenet = tl.layers.LambdaLayer(imagenet, fn=keras_block, name='kerasImage')
print(imagenet.outputs.shape)




[TL] InputLayer  image_input: (1, 5, 640, 480, 3)
[TL] LambdaLayer  kerasImage
Instructions for updating:
Colocations handled automatically by placer.
(1, 5, 80, 60, 1)


In [ ]:
imagenet = tf.compat.v2.keras.layers.ConvLSTM2D(filters=64,kernel_size=[5,5],strides=[2,2,1],padding='same',activation=tf.nn.relu,return_sequences=True)(imagenet)
    imagenet = tf.keras.layers.BatchNormalization()(imagenet)
    imagenet = tf.compat.v2.keras.layers.ConvLSTM2D(filters=64,kernel_size=[5,5],strides=[2,2,1],padding='same',activation=tf.nn.relu,return_sequences=True)(imagenet)
    imagenet = tf.keras.layers.BatchNormalization()(imagenet)
    imagenet = tf.compat.v2.keras.layers.ConvLSTM2D(filters=64,kernel_size=[5,5],strides=[2,2,1],padding='same',activation=tf.nn.relu,return_sequences=True)(imagenet)
    imagenet = tf.keras.layers.BatchNormalization()(imagenet)
    imagenet = tf.keras.layers.Conv3D(filters=1, kernel_size=(3, 3, 3),activation='sigmoid',padding='same')(imagenet)

In [5]:
obs_shape = [3,512,640, 480,15]
scan = tf.placeholder(tf.float32, [None, obs_shape[1], obs_shape[0]],'scan_ph')
goal = tf.placeholder(tf.float32, [None, 2], 'goal_ph')
vel  = tf.placeholder(tf.float32, [None, 2], 'vel_ph')

scannet = tl.layers.InputLayer(scan, name='scan_input')
scannet = tl.layers.MeanPool1d(scannet, filter_size=3, strides=2, name='min_pooling1')
print(scannet.outputs.shape)
scannet = tl.layers.MeanPool1d(scannet, filter_size=3, strides=2, name='min_pooling2')
print(scannet.outputs.shape)
scannet = tl.layers.MeanPool1d(scannet, filter_size=3, strides=2, name='min_pooling3')
print(scannet.outputs.shape)
scannet = tl.layers.Conv1dLayer(scannet, act=tf.nn.relu, shape=[5, obs_shape[0], 8], stride=2,name='cnn1')
print(scannet.outputs.shape)
scannet = tl.layers.Conv1dLayer(scannet, act=tf.nn.relu, shape=[3, 8, 16], stride=2,name='cnn2')
print(scannet.outputs.shape)
scannet = tl.layers.FlattenLayer(scannet, name='fl')
print(scannet.outputs.shape)
scannet = tl.layers.DenseLayer(scannet, n_units=128, act=tf.nn.relu, name='cnn_output')
scan_output = scannet.outputs


act_net = tl.layers.InputLayer(tf.concat([goal, vel, scan_output, image_output], axis=1), name='goal_input')
act_net = tl.layers.DenseLayer(act_net, n_units=64, act=tf.nn.tanh, name='act1')
act_net = tl.layers.DenseLayer(act_net, n_units=64, act=tf.nn.tanh, name='act2')

linear = tl.layers.DenseLayer(act_net, n_units=1, act=tf.nn.sigmoid, name='linear')
angular = tl.layers.DenseLayer(act_net, n_units=1, act=tf.nn.tanh, name='angular')


[TL] InputLayer  scan_input: (?, 512, 3)
[TL] MeanPool1d min_pooling1: filter_size: 3 strides: 2 padding: valid
Instructions for updating:
Use keras.layers.average_pooling1d instead.
(?, 255, 3)
[TL] MeanPool1d min_pooling2: filter_size: 3 strides: 2 padding: valid
(?, 127, 3)
[TL] MeanPool1d min_pooling3: filter_size: 3 strides: 2 padding: valid
(?, 63, 3)
[TL] Conv1dLayer cnn1: shape: [5, 3, 8] stride: 2 pad: SAME act: relu
(?, 32, 8)
[TL] Conv1dLayer cnn2: shape: [3, 8, 16] stride: 2 pad: SAME act: relu
(?, 16, 16)
[TL] FlattenLayer fl: 256
(?, 256)
[TL] DenseLayer  cnn_output: 128 relu


NameError: name 'image_output' is not defined